In [5]:
import math

In [8]:
def calc_euclidean_distance(row1, row2):
    if (len(row1) != len(row2)):
        raise Exception("Something is wrong with your data! Both rows are not the same size")
    distance = 0.0
    for i in range(len(row1)-1):
        # assume that last row is value we want to predict
        # use for loop so that we can scale for higher dimensions as well
        distance = distance + (row1[i] - row2[i]) ** 2
    return math.sqrt(distance)

In [12]:
def get_most_similar_neighbors(train, test, k):
    distances = []
    for row in train:
        curr_dist = calc_euclidean_distance(row, test)
        distances.append((row, curr_dist))
    distances.sort(key = lambda x : x[1]) #sorting by distance
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][0])
    return neighbors

In [13]:
def test():
    dataset = [[2.7810836,2.550537003,0],
        [1.465489372,2.362125076,0],
        [3.396561688,4.400293529,0],
        [1.38807019,1.850220317,0],
        [3.06407232,3.005305973,0],
        [7.627531214,2.759262235,1],
        [5.332441248,2.088626775,1],
        [6.922596716,1.77106367,1],
        [8.675418651,-0.242068655,1],
        [7.673756466,3.508563011,1]]

#     row0 = dataset[0]
#     for row in dataset:
#         distance = calc_euclidean_distance(row0, row)
#         print(distance)

    neighbors = get_most_similar_neighbors(dataset, dataset[0], 3)
    for neighbor in neighbors:
        print(neighbor)

test()

[2.7810836, 2.550537003, 0]
[3.06407232, 3.005305973, 0]
[1.465489372, 2.362125076, 0]
